### ELO Custom Rating

#### Libraries

In [176]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(lplyr,warn.conflicts=F)
library(tidyr,warn.conflicts=F)
library(tidyimpute,warn.conflicts=F)

library(psych,warn.conflicts=F)
library(ggplot2,warn.conflicts=F)

#### Load 2011+

In [177]:
# load the stats
data.regulars <- readRDS('../datafiles_rds/RegularSeasonDetailedResults.rds')
# keep 2011+
data.regulars %>% filter(Season>2010) -> data.regulars

In [178]:
# load the stats
data.teams <- readRDS('../datafiles_rds/Teams.rds')

data.teams %>%
    select(TeamID,TeamName) %>%
    mutate(TeamName=str_to_lower(TeamName), 
           TeamName=str_replace_all(TeamName,' ','_'), 
           TeamName=str_remove_all(TeamName,'&')) -> df.teams

#### Prepare data Win-Loss

In [192]:
data.regulars %>% 
    select(Season,DayNum,WTeamID,LTeamID,WLoc,WScore,LScore) %>% 
    mutate(WScorePct=(WScore+1)/(WScore+LScore+2)) -> df.regulars

In [193]:
# teams list
df.regulars %>% 
    select(Season,TeamID=WTeamID) %>% 
    bind_rows(dataset %>% select(Season,TeamID=LTeamID)) %>% 
    distinct() %>% 
    arrange(Season,TeamID) %>% 
    mutate(DayRating=0, Rating=0) -> df.ratings

#### Elo Rating

In [194]:
ComputeELO <- function(dataset,stat,K=32,Ksi=400) {
    stat_var <- enquo(stat)
    dataset %>% rename(Statistic := !!stat_var) -> dataset
    for(i in 1:140) {
       dataset %>% 
            filter(DayNum==i) %>% 
            inner_join(df.ratings,by=c('Season','WTeamID'='TeamID')) %>% 
            rename(DayRating.x=DayRating,Rating.x=Rating) %>%
            arrange(desc(DayRating.x)) %>% 
            group_by(Season,WTeamID) %>% 
            slice(1:1) %>% 
            ungroup() %>%
            inner_join(df.ratings,by=c('Season','LTeamID'='TeamID')) %>% 
            rename(DayRating.y=DayRating,Rating.y=Rating) %>%
            arrange(desc(DayRating.y)) %>% 
            group_by(Season,LTeamID) %>% 
            slice(1:1) %>% 
            ungroup() %>%
            mutate(Mu=1/(1+10^((Rating.y-Rating.x)/Ksi))) %>%
            mutate(Delta.x=(Statistic-Mu)*K, Delta.y=-Delta.x) %>%
            mutate(NRating.x=Rating.x+Delta.x, NRating.y=Rating.y+Delta.y) -> tmp01
        tmp01 %>% select(Season,DayNum,TeamID=WTeamID,DayRating=DayNum,Rating=NRating.x) %>%
            bind_rows(tmp01 %>% select(Season,DayNum,TeamID=LTeamID,DayRating=DayNum,Rating=NRating.y)) -> tmp02
        df.ratings %>% bind_rows(tmp02) ->> df.ratings
    }    
}

In [198]:
ComputeELO(df.regulars,WScorePct)

In [199]:
df.ratings %>% 
    arrange(desc(DayRating)) %>% 
    group_by(Season,TeamID) %>% 
    slice(1:1) %>% 
    ungroup() -> df.ratings.last
df.ratings.last %>% arrange(desc(Rating)) %>% inner_join(df.teams,by='TeamID') %>% head

Season,TeamID,DayRating,Rating,TeamName
2017,1211,127,48.45174,gonzaga
2015,1246,132,47.73722,kentucky
2014,1257,131,47.27878,louisville
2015,1112,131,43.99447,arizona
2013,1196,132,41.39137,florida
2013,1257,131,41.11271,louisville


### Tourney matchups performance (validation dataset)

#### Load test data

In [200]:
data.valid.matchups <- readRDS('../datafiles_rds/TourneyMatchups.rds')
data.valid.matchups %>% filter(Season>2010) -> data.valid.matchups
data.valid.truth <- readRDS('../datafiles_rds/TourneyTrueResults.rds')

#### prepare

In [201]:
# matchups dataframe : all the possible matches between the 68 teams per Season 
data.valid.matchups %>%
    inner_join(df.ratings.last,by=c('Season','TeamID.x'='TeamID')) %>%
    select(-DayRating) %>%
    inner_join(df.ratings.last,by=c('Season','TeamID.y'='TeamID')) %>% 
    select(-DayRating) -> df.valid.matchups

In [202]:
df.valid.matchups %>% sample_n(5)

Season,TeamID.x,TeamID.y,Rating.x,Rating.y
2017,1166,1235,19.02739,23.204568
2018,1242,1393,24.48035,10.579117
2017,1374,1436,36.17314,25.510492
2016,1139,1344,17.76325,12.567446
2012,1143,1443,21.02749,-1.676241


In [203]:
# machups predictions
K=32
Ksi=400
df.valid.matchups %>% 
    mutate(ID=str_c(Season,'_',TeamID.x,'_',TeamID.y)) %>% 
    mutate(Prob= 1/(1+10^((Rating.y-Rating.x)/Ksi)) ) %>% 
    mutate(Pred=ifelse(Prob>0.5,1,0)) %>% 
    mutate(Prob=round(Prob,3)) %>%
    mutate_which(Prob>0.95,Prob=0.95) %>%
    mutate_which(Prob<0.05,Prob=0.05) %>%
    select(Season,ID,Pred,Prob) -> df.valid.submit
df.valid.submit %>% sample_n(5)

Season,ID,Pred,Prob
2018,2018_1438_1439,1,0.530
2013,2013_1166_1207,0,0.495
2012,2012_1104_1436,1,0.507
2012,2012_1124_1254,1,0.526
2012,2012_1355_1443,1,0.534


In [205]:
# merge prediction and truth
data.valid.truth %>% inner_join(df.valid.submit,by='ID') -> df.valid.results

In [206]:
# compute accuracy & Logloss per Season
df.valid.results %>% 
    mutate(OK=ifelse(Pred==Target,1,0)) %>%
    mutate(LogLoss=Target*log(Prob) + (1-Target)*log(1-Prob)) %>%
    group_by(Season) %>%
    summarise(TC=sum(OK),N=n(),FC=N-TC,Acc=TC/N,LogLoss=-mean(LogLoss)) %>%
    select(Season,TC,FC,N,Acc,LogLoss) -> df.valid.perf
df.valid.perf %>% arrange(LogLoss) %>% group_by(Season) %>% slice(1:1) %>% ungroup() 

Season,TC,FC,N,Acc,LogLoss
2011,42,25,67,0.6268657,0.6810992
2012,48,19,67,0.7164179,0.6775226
2013,40,27,67,0.5970149,0.6769426
2014,44,23,67,0.6567164,0.6759818
2015,49,18,67,0.7313433,0.6684752
2016,46,21,67,0.6865672,0.6772046
2017,45,22,67,0.6716418,0.6741502
2018,48,19,67,0.7164179,0.6787731


In [208]:
df.valid.results %>% saveRDS('ELO_results.rds')